<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Fine_Tuning_NSQL350.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from builtins import print
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
    PeftModel,
    PeftConfig,
    prepare_model_for_kbit_training
)
from sympy import principal_branch
from torch.distributed.autograd import context
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, pipeline
import pandas as pd
from datasets import load_dataset, Dataset
import json
import re

os.environ["WANDB_MODE"] ="offline"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

model_final ="model/nsql-final"
device= "cuda:0"
model = AutoModelForCausalLM.from_pretrained(model_final, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map=device,
                                             trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_final)

def generateQuery(user_question, user_context, m_new_tokens):
    eval_prompt = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You must output the SQL query that answers the question.
### Input:
{user_question}

### Context:
{user_context}
### Response:

""".strip()
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    model.eval()
    text = tokenizer.decode(model.generate(**model_input, max_new_tokens=m_new_tokens)[0], skip_special_tokens=True)
    l = len(text)
    p_start = text.find('### Response:')
    p_start = p_start + len('### Response:')
    strOutput = ""
    for i in range(p_start + 1, l):
        code = ord(text[i])
        if code == 10:
            p = i
            break
    strOutput= text[p_start:p]
    return strOutput, text

user_question="What is the production today of the Assy V3 phase?"
user_context="CREATE TABLE tblMovingV3(MOVV3_ID uniqueidentifier,CURRENTSITE varchar,ACTIVITY nvarchar,EVENDATE nvarchar,EQPID nvarchar,STEPID nvarchar, PRODUCTID nvarchar,MOVING numeric, V3_WEEK int, V3_MONTH int,V3_YEAR int,V3_DAY int); CREATE TABLE tblEQPList(EQPL_ID uniqueidentifier,MODULE nvarchar,STEPID nvarchar,PROCESS nvarchar,SUB_PROCESS nvarchar, VENDOR nvarchar,EQP_TYPE nvarchar)"

text1, text2 = generateQuery(user_question = user_question, user_context = user_context,  m_new_tokens=1000)
print(text1)
print('-------------------------------------------------')
print(text2)

